# Naive Bayes (the easy way)

We'll cheat by using sklearn.naive_bayes to train a spam classifier! Most of the code is just loading our training data into a pandas DataFrame that we can play with:

In [77]:
import os
import io
import numpy as np
from pandas import DataFrame
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.naive_bayes import MultinomialNB

def readFiles(path):
    for root, dirnames, filenames in os.walk(path):
        for filename in filenames:
            path = os.path.join(root, filename)

            inBody = False
            lines = []
            f = io.open(path, 'r', encoding='latin1')
            for line in f:
                if inBody:
                    lines.append(line)
                elif line == '\n':
                    inBody = True
            f.close()
            message = '\n'.join(lines)
            yield path, message


def dataFrameFromDirectory(path, classification):
    rows = []
    index = []
    for filename, message in readFiles(path):
        rows.append({'message': message, 'class': classification})
        index.append(filename)

    return DataFrame(rows, index=index)

data = DataFrame({'message': [], 'class': []})

data = data.append(dataFrameFromDirectory('./emails/spam', 'spam'))
data = data.append(dataFrameFromDirectory('./emails/ham', 'ham'))


Let's have a look at that DataFrame:

In [ ]:
data.head()

Now we will use a CountVectorizer to split up each message into its list of words, and throw that into a MultinomialNB classifier. Call fit() and we've got a trained spam filter ready to go! It's just that easy.

In [ ]:
vectorizer = CountVectorizer()
counts = vectorizer.fit_transform(data['message'].values)

print(counts[1])

classifier = MultinomialNB()
targets = data['class'].values
classifier.fit(counts, targets)

Let's try it out:

In [ ]:
examples = ['Free Viagra now!!!', "Hi Bob, how about a free game of golf tomorrow?"]
example_counts = vectorizer.transform(examples)
print(example_counts)
predictions = classifier.predict(example_counts)
predictions

## Activity

Our data set is small, so our spam classifier isn't actually very good. Try running some different test emails through it and see if you get the results you expect.

If you really want to challenge yourself, try applying train/test to this spam classifier - see how well it can predict some subset of the ham and spam emails.

In [61]:
data.head()

# split data into train and test

msk = np.random.rand(len(data)) < 0.8

x_train = counts[msk]
x_test = counts[~msk]

y_train = targets[msk]
y_test = targets[~msk]

classifier.fit(x_train, y_train)



MultinomialNB()

In [66]:
predictions = classifier.predict(x_test)

In [76]:
correct = 0
for i in range(len(predictions)):
    if predictions[i] == y_test[i]:
        correct += 1
        
print(f'Accuracy: {correct / float(len(predictions))}')

Accuracy: 0.9644670050761421


In [106]:
from sklearn.metrics import accuracy_score, recall_score, fbeta_score
print('Accuracy:', accuracy_score(y_test, predictions))
print('Recall Score (Ham):', recall_score(y_test, predictions, pos_label='ham'))
print('Recall Score (Spam):', recall_score(y_test, predictions, pos_label='spam'))
print('FBeta Score (Ham):', fbeta_score(y_test, predictions, beta=1, pos_label='ham'))
print('FBeta Score (Spam):', fbeta_score(y_test, predictions, beta=1, pos_label='spam'))

Accuracy: 0.9644670050761421
Recall Score (Ham): 0.9918200408997955
Recall Score (Spam): 0.8333333333333334
FBeta Score (Ham): 0.9788092835519677
FBeta Score (Spam): 0.8900523560209425
